In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import pandas as pd
pd.set_option('precision', 2)
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import re
import time
from pprint import pprint

### sklearn Pre-processing
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.exceptions import DataConversionWarning

### sklearn Metrics
from sklearn import metrics
from sklearn.metrics import silhouette_score
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, make_scorer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### sklearn models
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor

### Others
import nltk
from nltk.stem import WordNetLemmatizer
from subprocess import check_output



In [3]:
#pwd()

In [4]:
raw_interactions = pd.read_csv('/home/zanguyen/DSC 180B/data/RAW_interactions.csv')
raw_recipes = pd.read_csv('/home/zanguyen/DSC 180B/data/RAW_recipes.csv')

In [5]:
# PP_recipes = pd.read_csv('/home/zanguyen/DSC 180B/data/PP_recipes.csv')
# PP_users = pd.read_csv('/home/zanguyen/DSC 180B/data/PP_users.csv')

In [6]:

raw_recipes = raw_recipes.astype({'id': 'object','contributor_id': 'object'})
raw_recipes = raw_recipes.rename(columns={'id':'recipe_id'})
raw_recipes = raw_recipes.set_index('recipe_id')
#raw_recipes = raw_recipes.set_index('id')

In [7]:
raw_recipes.head()

,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
recipe_id,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
31490,a bit different breakfast pizza,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
112140,all in the kitchen chili,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
59389,alouette potatoes,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
44061,amish tomato ketchup for canning,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [8]:
raw_recipes.describe()

,minutes,n_steps,n_ingredients
count,2.32e+05,231637.00,231637.00
mean,9.40e+03,9.77,9.05
std,4.46e+06,6.00,3.73
min,0.00e+00,0.00,1.00
25%,2.00e+01,6.00,6.00
50%,4.00e+01,9.00,9.00
75%,6.50e+01,12.00,11.00
max,2.15e+09,145.00,43.00


In [9]:
raw_interactions.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [10]:
raw_interactions[['recipe_id','rating']].groupby('recipe_id',as_index=False)['rating'].agg(['mean','count']).rename(columns={"mean": "mean_rating", "count": "review_count"})


,mean_rating,review_count
recipe_id,,
38,4.25,4
39,3.00,1
40,4.33,9
41,4.50,2
43,1.00,1
...,...,...
537459,5.00,1
537485,5.00,1
537543,0.00,1


In [11]:
interactions = raw_interactions[['rating','recipe_id']].groupby('recipe_id')['rating'].agg(['mean','count']).rename(columns = {'mean':'mean_rating','count':'review_count'})









In [12]:
raw_data = raw_recipes.merge(interactions, on = 'recipe_id')
raw_data.head()


,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,mean_rating,review_count
recipe_id,,,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,5.0,3
31490,a bit different breakfast pizza,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,3.5,4
112140,all in the kitchen chili,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,4.0,1
59389,alouette potatoes,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,4.5,2
44061,amish tomato ketchup for canning,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,5.0,1


In [13]:
raw_data.ingredients

recipe_id
137739    ['winter squash', 'mexican seasoning', 'mixed ...
31490     ['prepared pizza crust', 'sausage patty', 'egg...
112140    ['ground beef', 'yellow onions', 'diced tomato...
59389     ['spreadable cheese with garlic and herbs', 'n...
44061     ['tomato juice', 'apple cider vinegar', 'sugar...
                                ...                        
486161    ['celery', 'onion', 'green sweet pepper', 'gar...
493372    ['paprika', 'salt', 'garlic powder', 'onion po...
308080    ['hard-cooked eggs', 'mayonnaise', 'dijon must...
298512    ['butter', 'eagle brand condensed milk', 'ligh...
298509    ['granulated sugar', 'shortening', 'eggs', 'fl...
Name: ingredients, Length: 231637, dtype: object

In [14]:
display((raw_data.ingredients.iloc[0]))
(type((raw_data.ingredients.iloc[0])))

"['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']"

str

In [15]:
raw_data.head()

,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,mean_rating,review_count
recipe_id,,,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,5.0,3
31490,a bit different breakfast pizza,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,3.5,4
112140,all in the kitchen chili,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,4.0,1
59389,alouette potatoes,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,4.5,2
44061,amish tomato ketchup for canning,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,5.0,1


___Need to convert ingredient strings to list___

In [16]:
((raw_data.ingredients.iloc[0])[1:-1].replace("'", "")).strip()

'winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt'

In [17]:
(list((raw_data.ingredients.iloc[0])[1:-1].replace("'", "").split(",")))

['winter squash',
 ' mexican seasoning',
 ' mixed spice',
 ' honey',
 ' butter',
 ' olive oil',
 ' salt']

In [18]:
raw_data

,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,mean_rating,review_count
recipe_id,,,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,5.0,3
31490,a bit different breakfast pizza,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,3.5,4
112140,all in the kitchen chili,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,4.0,1
59389,alouette potatoes,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,4.5,2
44061,amish tomato ketchup for canning,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486161,zydeco soup,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22,5.0,6
493372,zydeco spice mix,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13,5.0,1
308080,zydeco ya ya deviled eggs,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8,5.0,5


In [19]:
train = pd.read_json('/home/zanguyen/DSC 180B/data/train.json')
test = pd.read_json('/home/zanguyen/DSC 180B/data/test.json')

In [20]:
train.cuisine.nunique()

20

In [21]:
train.head()

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


In [22]:
(train['ingredients'].iloc[0])

['romaine lettuce',
 'black olives',
 'grape tomatoes',
 'garlic',
 'pepper',
 'purple onion',
 'seasoning',
 'garbanzo beans',
 'feta cheese crumbles']

In [23]:
# train['ingredient_list'] = [','.join(z).strip() for z in train['ingredients']]

# ' '.join([str(elem) for elem in s]) 


In [24]:
def to_str(z):
    z = ','.join(z)
    return z

In [25]:
train['ingredient_list'] = train.ingredients.apply(to_str)

In [26]:
train.head()

,id,cuisine,ingredients,ingredient_list
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...","romaine lettuce,black olives,grape tomatoes,ga..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...","plain flour,ground pepper,salt,tomatoes,ground..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...","eggs,pepper,salt,mayonaise,cooking oil,green c..."
3,22213,indian,"[water, vegetable oil, wheat, salt]","water,vegetable oil,wheat,salt"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...","black pepper,shallots,cornflour,cayenne pepper..."


In [ ]:
#vectorizer = TfdfVectorizer()

In [38]:
ingredients = train['ingredient_list']
tfidf_matrix= TfidfVectorizer().fit_transform(ingredients).todense()
cuisines = train['cuisine']

In [28]:
train

,id,cuisine,ingredients,ingredient_list
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...","romaine lettuce,black olives,grape tomatoes,ga..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...","plain flour,ground pepper,salt,tomatoes,ground..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...","eggs,pepper,salt,mayonaise,cooking oil,green c..."
3,22213,indian,"[water, vegetable oil, wheat, salt]","water,vegetable oil,wheat,salt"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...","black pepper,shallots,cornflour,cayenne pepper..."
...,...,...,...,...
39769,29109,irish,"[light brown sugar, granulated sugar, butter, ...","light brown sugar,granulated sugar,butter,warm..."
39770,11462,italian,"[KRAFT Zesty Italian Dressing, purple onion, b...","KRAFT Zesty Italian Dressing,purple onion,broc..."
39771,2238,irish,"[eggs, citrus fruit, raisins, sourdough starte...","eggs,citrus fruit,raisins,sourdough starter,fl..."
39772,41882,chinese,"[boneless chicken skinless thigh, minced garli...","boneless chicken skinless thigh,minced garlic,..."


In [39]:
tfidf_matrix


matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
cuisines.head()

0          greek
1    southern_us
2       filipino
3         indian
4         indian
Name: cuisine, dtype: object

In [41]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, cuisines, test_size=0.2)




In [42]:
#param_grid = {'n_estimators': [100]}

grid = GridSearchCV(RandomForestClassifier(), {'n_estimators': [100]}, cv=5)

In [43]:
grid.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [44]:

grid.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [45]:
grid.score(X_test, y_test)

0.7529855436832181

In [52]:

y_predictions = grid.predict(X_test)


In [53]:
print(accuracy_score(y_test, y_predictions))
cuisines = train['cuisine'].value_counts().index

0.7529855436832181


In [57]:
raw_data['ingredient_list'] = [''.join(z).strip() for z in raw_data['ingredients']]
raw_data_ingredients = raw_data['ingredient_list']

In [59]:






raw_data_tfidf_matrix = vectorizer.transform(raw_data_ingredients)


raw_data_cuisines = grid.predict(raw_data_tfidf_matrix)


raw_data['cuisine'] = raw_data_cuisines


raw_data.head()

,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,mean_rating,review_count,ingredient_list,cuisine
recipe_id,,,,,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,5.0,3,"['winter squash', 'mexican seasoning', 'mixed ...",mexican
31490,a bit different breakfast pizza,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,3.5,4,"['prepared pizza crust', 'sausage patty', 'egg...",italian
112140,all in the kitchen chili,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,4.0,1,"['ground beef', 'yellow onions', 'diced tomato...",mexican
59389,alouette potatoes,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,4.5,2,"['spreadable cheese with garlic and herbs', 'n...",italian
44061,amish tomato ketchup for canning,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,5.0,1,"['tomato juice', 'apple cider vinegar', 'sugar...",southern_us


In [35]:
raw_data[['name','minutes','nutrition','n_steps','steps','description','ingredients','n_ingredients','mean_rating','ingredient_list','cuisine']].iloc[44:].head(5)







,name,minutes,nutrition,n_steps,steps,description,ingredients,n_ingredients,mean_rating,ingredient_list,cuisine
recipe_id,,,,,,,,,,,
33606,italian sandwich pasta salad,25,"[279.2, 14.0, 12.0, 22.0, 23.0, 11.0, 12.0]",6,"['cook pasta and set aside', 'place onions , d...",this is a fun salad that uses the same ingredi...,"['tri-color spiral pasta', 'dill pickles', 'ri...",16,5.00,"['tri-color spiral pasta', 'dill pickles', 'ri...",italian
94710,italian fries,15,"[241.0, 12.0, 1.0, 23.0, 10.0, 11.0, 12.0]",4,['deep fry french fries according to package d...,"ok, so this is not *really* italian, but it is...","['frozen french fries', 'oil', 'salt & freshly...",7,5.00,"['frozen french fries', 'oil', 'salt & freshly...",italian
35173,italian gut busters,45,"[816.9, 100.0, 11.0, 97.0, 98.0, 180.0, 2.0]",7,['lay out sandwich rolls on jelly roll pans / ...,my sister-in-law made these for us at a family...,"['sandwich bun', 'good seasonings italian sala...",9,4.00,"['sandwich bun', 'good seasonings italian sala...",italian
83025,jeanne s style birthday cake,230,"[5467.4, 516.0, 1196.0, 135.0, 110.0, 615.0, 1...",25,"['to prepare base , cut shortening into dry in...",a bakery in winnipeg is famous for this specia...,"['shortening', 'icing sugar', 'vanilla', 'all-...",10,2.29,"['shortening', 'icing sugar', 'vanilla', 'all-...",southern_us
52804,jiffy extra moist carrot cake,50,"[612.1, 49.0, 170.0, 25.0, 15.0, 39.0, 25.0]",8,"['preheat oven to 350 degrees', 'mix together ...","this is a very tasty, moist, carrot cake. a ni...","['yellow cake mix', 'vanilla instant pudding m...",11,4.00,"['yellow cake mix', 'vanilla instant pudding m...",southern_us


In [36]:
#raw_data.to_csv('/home/zanguyen/DSC 180B/final_data.csv')

In [37]:
raw_data.cuisine.nunique()

20